<a href="https://colab.research.google.com/github/jubeam/my_js_project/blob/master/CNN_%EC%8B%A4%EC%8A%B5_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:


from keras.preprocessing.image import ImageDataGenerator

from google.colab import drive
drive.mount('/gdrive', force_remount=True)



img_gen = ImageDataGenerator(
        rescale=1/255,         # 픽셀 값을 0~1 범위로 변환
        rotation_range=40,      # 40도까지 회전
        width_shift_range=0.2,  # 20%까지 좌우 이동
        height_shift_range=0.2, # 20%까지 상하 이동
        shear_range=0.2,        # 20%까지 기울임
        zoom_range=0.2,         # 20%까지 확대
        horizontal_flip=True,   # 좌우 뒤집기
    )

train = img_gen.flow_from_directory(
    '/gdrive/My Drive/Colab Notebooks/train',     # 이미지 디렉토리
    target_size=(2100,, 2100), # 변환할 크기는 가로 100, 세로 100
    color_mode='grayscale',       # 컬러는 rgb, 흑백은 grayscale. 생략하면 컬러로 처리한다
    class_mode='binary')    # 고양이 vs. 개로 binary 분류


valid = ImageDataGenerator(rescale=1/255).flow_from_directory(
    '/gdrive/My Drive/Colab Notebooks/validation',
    target_size=(2100, 2100),
     color_mode='grayscale'
    class_mode='binary',
    shuffle=False)

#모형 만들기
from keras.layers import Conv2D, Dense, Dropout, Flatten, MaxPooling2D
from keras.models import Sequential

model = Sequential()
model.add(Conv2D(32, (5, 5), activation='relu', input_shape=(2100,2100)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.summary()

from keras.optimizers import Adam, RMSprop
from keras.callbacks import TensorBoard
LOG_DIR = 'log_model'
model.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer=Adam())
from keras.callbacks import ModelCheckpoint, TensorBoard
model.fit_generator(
    train, validation_data=valid,steps_per_epoch=10, validation_steps=5, epochs=5,
    callbacks=[
        TensorBoard(log_dir=LOG_DIR),
        ModelCheckpoint('cnn-{epoch:02d}.hdf5', save_best_only=True),
    ])

from keras.preprocessing.image import load_img 
img = load_img('/gdrive/My Drive/Colab Notebooks/cancer/file10.jpeg', target_size=(2100, 2100))
import numpy
img_array = numpy.array(img) / 255
img_array.shape
model.predict(numpy.array([img_array]))


!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -o ngrok-stable-linux-amd64.zip
get_ipython().system_raw(f'tensorboard --logdir {LOG_DIR} --host 0.0.0.0 --port 6006 &')
get_ipython().system_raw('./ngrok http 6006 &')
! curl -s http://localhost:4040/api/tunnels | tr '"' '\n' | grep "https://"

Mounted at /gdrive
Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 96, 96, 32)        2432      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 48, 48, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 46, 46, 32)        9248      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 23, 23, 32)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 16928)             0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 16928)             0         
_____________________________________________

/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:434: RuntimeWarning: Can save best model only with val_loss available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)


--2019-08-25 01:08:47--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.203.102.189, 52.72.245.79, 3.220.125.240, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.203.102.189|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13607069 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  12.98M  77.1MB/s    in 0.2s    

2019-08-25 01:08:48 (77.1 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13607069/13607069]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
https://a27633c4.ngrok.io
